In [1]:
import pandas as pd
import numpy as np
import tarfile
import os
import glob as glob

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 

In [2]:
os.getcwd()

'/home/thanish/Competition/Aicrowd/notebooks'

In [3]:
# Unzipping the files and writing to directory

#This is a one time thing
# for file in glob.glob("../data/*.bz2"):
#     print(file)
#     tar = tarfile.open(file)
#     tar.extractall(path='../data/')

In [4]:
def check_null_values(DF):
    '''
    This function is to check the NULL values in each of the aircraft
    '''
    
    temp = DF.groupby(['aircraft']).agg({'latitude': lambda x: x.notnull().sum(),
                                         'longitude': lambda x: x.isnull().sum()}).reset_index()
    temp.rename(columns = {'latitude' : 'Not_null', 'longitude':'Null'}, inplace = True)
    print(temp.loc[temp.Null !=0,:])
    
    return temp.aircraft[temp.Null !=0].values.tolist(), temp.aircraft[temp.Null ==0].values.tolist()

In [5]:
# Reading each file 

pred_aircraft = []
train_aircraft = []
final_DF = pd.DataFrame()
final_train_sensor = pd.DataFrame()
for i, root_folder in enumerate(glob.glob("../data/training_*category_1/")):
    print(root_folder)
    print('file_' + str(i+1))
    filename = 'file_' + str(i+1)
    for file in glob.glob(root_folder + "/training_*"):
        print("Reading the file:", file)
        DF = pd.read_csv(file)
        DF['file'] = filename
        final_DF = final_DF.append(DF)
        print(final_DF.shape, DF.shape)

    for file in glob.glob(root_folder + "/sensors*"):
        print("Reading the file:", file)
        DF = pd.read_csv(file)
        #DF['file'] = filename
        final_train_sensor = final_train_sensor.append(DF)
        print(final_train_sensor.shape, DF.shape)
        print("")
        
print("Reading the test file")
test_DF = pd.read_csv('../data/round1_competition.csv')
final_test_sensor = pd.read_csv("../data/sensors.csv")

../data/training_2_category_1/
file_1
Reading the file: ../data/training_2_category_1/training_2_category_1.csv
(1887990, 10) (1887990, 10)
Reading the file: ../data/training_2_category_1/sensors.csv
(523, 5) (523, 5)

../data/training_3_category_1/
file_2
Reading the file: ../data/training_3_category_1/training_3_category_1.csv
(3890837, 10) (2002847, 10)
Reading the file: ../data/training_3_category_1/sensors.csv
(1046, 5) (523, 5)

../data/training_4_category_1/
file_3
Reading the file: ../data/training_4_category_1/training_4_category_1.csv
(5885427, 10) (1994590, 10)
Reading the file: ../data/training_4_category_1/sensors.csv
(1569, 5) (523, 5)

../data/training_1_category_1/
file_4
Reading the file: ../data/training_1_category_1/training_1_category_1.csv
(7959621, 10) (2074194, 10)
Reading the file: ../data/training_1_category_1/sensors.csv
(2092, 5) (523, 5)

../data/training_5_category_1/
file_5
Reading the file: ../data/training_5_category_1/training_5_category_1.csv
(9911498,

In [6]:
final_DF

,id,timeAtServer,aircraft,latitude,longitude,baroAltitude,geoAltitude,numMeasurements,measurements,file
0,1,0.000,104,48.783554,8.625324,10972.80,11262.36,3,"[[463,947958973,4],[440,948088205,35],[254,947...",file_1
1,2,0.004,3037,50.348340,8.447373,11277.60,11506.20,3,"[[101,987783546,55],[440,987663030,42],[125,98...",file_1
2,3,0.006,1103,45.161911,10.036534,4099.56,4312.92,2,"[[113,984673796,91],[206,985022796,95]]",file_1
3,4,0.006,2934,49.256561,2.781161,11277.60,11574.78,3,"[[314,995226531,65],[115,996239734,45],[412,99...",file_1
4,5,0.007,1355,45.517614,8.450898,3352.80,3573.78,2,"[[113,993560765,88],[206,993366328,105]]",file_1
...,...,...,...,...,...,...,...,...,...,...
1930133,1930134,3599.981,1016,46.206528,11.765487,10363.20,10965.18,3,"[[188,3600968386718,17],[130,3600968056953,91]...",file_7
1930134,1930135,3599.984,823,52.156382,7.402666,11887.20,12435.84,3,"[[238,3600949521359,76],[394,3600949645281,72]...",file_7
1930135,1930136,3599.985,1676,45.325633,12.803159,10980.42,11620.50,2,"[[114,3600954774859,49],[130,3600954548015,67]]",file_7
1930136,1930137,3599.990,639,47.856674,11.377716,10363.20,10927.08,2,"[[130,3600959947312,33],[131,3600959188562,111]]",file_7


In [7]:
# Fixing the train and test sensors

print("Train sensor", final_train_sensor.shape)
final_train_sensor = final_train_sensor.drop_duplicates()
print("Unique train sensor", final_train_sensor.shape)

final_train_sensor.rename(columns = {"serial" : "sensor_id",
                                     "latitude": "sensor_lat",
                                     "longitude": "sensor_lon"}, inplace = True)

final_test_sensor.rename(columns = {"serial" : "sensor_id",
                                     "latitude": "sensor_lat",
                                     "longitude": "sensor_lon"}, inplace = True)

Train sensor (3661, 5)
Unique train sensor (523, 5)


In [8]:
# Separating the Null and not-Null train and test sets

train_DF = final_DF.loc[final_DF.latitude.notnull(), :].copy()
valid_DF = final_DF.loc[final_DF.latitude.isnull(), :].copy()
test_DF_not_null = test_DF.loc[test_DF.latitude.notnull()].copy()
test_DF_null = test_DF.loc[test_DF.latitude.isnull()].copy()

# del final_DF

train_aircraft = train_DF.aircraft.unique()
valid_aircraft = valid_DF.aircraft.unique()

test_not_null_aircraft  = test_DF_not_null.aircraft.unique()
test_null_aircraft  = test_DF_null.aircraft.unique()

print(train_DF.shape, valid_DF.shape, test_DF_not_null.shape, test_DF_null.shape)

(12994599, 10) (716624, 10) (1727256, 9) (109474, 9)


In [9]:
train_aircraft.shape, test_not_null_aircraft.shape, test_null_aircraft.shape

((2927,), (1603,), (32,))

In [14]:
##########################################################################################
# This methods is for converting the measurement column to multiple columns

max_cols = final_DF.numMeasurements.max()
def measurement_2_multicol(df):
    temp = df.copy()
    measurement = temp.measurements.str.split(",").apply(pd.Series)

    col = []
    for i in range(max_cols):
        col.extend(["sensor_id_" + str(i), "sensor_time_" + str(i), "sensor_signal_strength_" + str(i)])
    measurement.columns = col[0 : measurement.shape[1]]

    for column in  measurement.columns:
        measurement[column] = measurement[column].str.replace("[", "").str.replace("]", "").astype('float')
    df = pd.concat([temp, measurement], axis = 1)
    
    return df


In [15]:
##########################################################################################
# The methods if for converting the measurement column to multiple rows and merge with the sensor details

def DF_explode_merge(DF, sensor):
    DF.measurements = DF.measurements.str.split("\],\[")
    DF = DF.explode('measurements')
    print("Done Exploding")
    
    DF.measurements = DF.measurements.str.replace('\[',"").str.replace('\]',"")
    DF[['sensor_id', 'sensor_time', 'signal_strength']] = DF.measurements.str.split(",").apply(pd.Series).astype(float)
    DF.reset_index(drop = True, inplace = True)
    print("Done filling the sensor_id, time and strength")
    
    DF = pd.merge(DF, sensor, left_on = ['sensor_id'], right_on = ['sensor_id'], how = 'left')
    print("Done Merging")
    
    return DF

# print("Forming the train set")
# train_DF = DF_explode_merge(DF = train_DF, sensor = final_train_sensor)
# print("")
# print("Forming the test set")
# test_DF = DF_explode_merge(DF = test_DF_null, sensor = final_test_sensor)

In [16]:
method = 'column-wise'

if method == 'row-wise':
    for index, i in enumerate(range(0, len(train_aircraft), 500)):
        print("Running batch_{} {}:{}".format(index, i, i+500) )

        selected_aircraft = train_aircraft[i:i+500]
        temp = train_DF.loc[train_DF.aircraft.isin(selected_aircraft),].reset_index(drop = True).copy()

        print("Exploding batch {} the train set".format(index))
        temp = DF_explode_merge(DF = temp, sensor = final_train_sensor)

        filename = '../data/train_batch_' + str(index) + '_rowwise.pickle'
        print("Writing the file {}".format(filename))
        temp.to_pickle(filename)
        print("")
        
    temp = DF_explode_merge(DF = test_DF_null, sensor = final_test_sensor)

    filename = '../data/test_DF_null.pickle'
    print("Writing the file {}".format(filename))
    temp.to_pickle(filename)
    print("")
    
elif method == 'column-wise':
    for index, i in enumerate(range(0, len(train_aircraft), 500)):
        print("Running batch_{} {}:{}".format(index, i, i+500) )

        selected_aircraft = train_aircraft[i:i+500]
        temp = train_DF.loc[train_DF.aircraft.isin(selected_aircraft),].reset_index(drop = True).copy()

        print("Exploding batch {} the train set".format(index))
        temp = measurement_2_multicol(df = temp)

        filename = '../data/train_batch_' + str(index) + '_colwise.pickle'
        print("Writing the file {}".format(filename))
        temp.to_pickle(filename)
        print("")
        
    temp = measurement_2_multicol(df = test_DF_null)

    filename = '../data/test_DF_null_colwise.pickle'
    print("Writing the file {}".format(filename))
    temp.to_pickle(filename)
    print("")

Running batch_0 0:500
Exploding batch 0 the train set
Writing the file ../data/train_batch_0_colwise.pickle

Running batch_1 500:1000
Exploding batch 1 the train set
Writing the file ../data/train_batch_1_colwise.pickle

Running batch_2 1000:1500
Exploding batch 2 the train set
Writing the file ../data/train_batch_2_colwise.pickle

Running batch_3 1500:2000
Exploding batch 3 the train set
Writing the file ../data/train_batch_3_colwise.pickle

Running batch_4 2000:2500
Exploding batch 4 the train set
Writing the file ../data/train_batch_4_colwise.pickle

Running batch_5 2500:3000
Exploding batch 5 the train set
Writing the file ../data/train_batch_5_colwise.pickle

Writing the file ../data/test_DF_null_colwise.pickle

